In [62]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [63]:
!pip install pyvis

In [64]:
# Required Libraries
import numpy as np
import pandas as pd
import textwrap

#from pyBibX.base import pbx_probe
from google.colab import data_table
import datetime
from datetime import datetime
import networkx
from pyvis.network import Network
import glob
import pickle

from google.colab.data_table import DataTable
DataTable.max_columns = 100

In [65]:
# Loading data from the pickle file
with open("/content/drive/MyDrive/Projeto CNI - Produções/13. Patentes/dw_df.pkl", 'rb') as file:
    dw_df = pickle.load(file)

with open("/content/drive/MyDrive/Projeto CNI - Produções/13. Patentes/dw_wo_df.pkl", 'rb') as file:
    wo_df = pickle.load(file)

In [66]:
dw_df

,index,Patent Number,Title,Patent Assignee,Patent Assignee Matrix,circle_color,triangle_color,International Patent Classification,IPC4_Desc,Family Size
0,0,US11613715-B1; CA3165890-A1; US2023114439-A1,Producing renewable transportation fuels for ...,MARATHON PETROLEUM CO LP (MAOC-C),MAOC-C,blue,"rgba(255, 0, 0, 0.5)",C10L,Fuels not otherwise provided for; natural gas;...,3
1,0,US11613715-B1; CA3165890-A1; US2023114439-A1,Producing renewable transportation fuels for ...,MARATHON PETROLEUM CO LP (MAOC-C),MAOC-C,blue,"rgba(255, 0, 0, 0.5)",C10L,Fuels not otherwise provided for; natural gas;...,3
2,0,US11613715-B1; CA3165890-A1; US2023114439-A1,Producing renewable transportation fuels for ...,MARATHON PETROLEUM CO LP (MAOC-C),MAOC-C,blue,"rgba(255, 0, 0, 0.5)",C01B,Non-metallic elements; compounds thereof (ferm...,3
3,0,US11613715-B1; CA3165890-A1; US2023114439-A1,Producing renewable transportation fuels for ...,MARATHON PETROLEUM CO LP (MAOC-C),MAOC-C,blue,"rgba(255, 0, 0, 0.5)",C10G,Cracking hydrocarbon oils; production of liqui...,3
4,0,US11613715-B1; CA3165890-A1; US2023114439-A1,Producing renewable transportation fuels for ...,MARATHON PETROLEUM CO LP (MAOC-C),MAOC-C,blue,"rgba(255, 0, 0, 0.5)",C12P,Fermentation or enzyme-using processes to synt...,3
...,...,...,...,...,...,...,...,...,...,...
871,68,WO2023114825-A1; US2023192485-A1,Generating purified hydrogen product stream ...,KLEIN F J (KLEI-Individual),KLEI-Individual,blue,"rgba(255, 0, 0, 0.5)",C01B,Non-metallic elements; compounds thereof (ferm...,1
872,68,WO2023114825-A1; US2023192485-A1,Generating purified hydrogen product stream ...,KLEIN F J (KLEI-Individual),KLEI-Individual,blue,"rgba(255, 0, 0, 0.5)",C01B,Non-metallic elements; compounds thereof (ferm...,1
873,68,WO2023114825-A1; US2023192485-A1,Generating purified hydrogen product stream ...,KLEIN F J (KLEI-Individual),KLEI-Individual,blue,"rgba(255, 0, 0, 0.5)",C01B,Non-metallic elements; compounds thereof (ferm...,1
874,68,WO2023114825-A1; US2023192485-A1,Generating purified hydrogen product stream ...,KLEIN F J (KLEI-Individual),KLEI-Individual,blue,"rgba(255, 0, 0, 0.5)",C10K,Purifying or modifying the chemical compositio...,1


In [67]:
patents_per_enterprise_dw = dw_df.groupby(['Patent Assignee Matrix', 'International Patent Classification', 'IPC4_Desc', 'circle_color']).nunique()["Patent Number"].reset_index()
patents_per_enterprise_dw

,Patent Assignee Matrix,International Patent Classification,IPC4_Desc,circle_color,Patent Number
0,BEHR-Individual,B01D,Separation (separating solids from solids by w...,blue,1
1,BEHR-Individual,C01B,Non-metallic elements; compounds thereof (ferm...,blue,1
2,BEHR-Individual,C10K,Purifying or modifying the chemical compositio...,blue,1
3,BRID-C,B60C,Vehicle tyres; tyre inflation; tyre changing; ...,blue,1
4,BRID-C,C08K,Use of inorganic or non-macromolecular organic...,blue,1
...,...,...,...,...,...
135,XDEV-C,C25B,Electrolytic or electrophoretic processes for ...,blue,1
136,XDEV-C,H01L,Semiconductor devices not covered by class h10...,blue,1
137,XDEV-C,H02J,Circuit arrangements or systems for supplying ...,blue,1
138,XDEV-C,H02S,Generation of electric power by conversion of ...,blue,1


Gera o grafo de Assignees e IPCs com peso na aresta sendo o número de patentes publicadas no IPC.

In [68]:
grafo = Network(height='800px', width='100%', notebook=True, cdn_resources='in_line')

for index, row in patents_per_enterprise_dw.iterrows():
    grafo.add_node(str(row["Patent Assignee Matrix"]), title=str(row["Patent Assignee Matrix"]),
                   color=row['circle_color'])
    grafo.add_node(str(row["International Patent Classification"]), title=str(row["IPC4_Desc"]), color='red', shape='box')

    # Add edge with weight as an attribute
    grafo.add_edge(str(row["Patent Assignee Matrix"]), str(row["International Patent Classification"]), weight=row["Patent Number"], title=str(row["Patent Number"]), width=row["Patent Number"])

# Configure the graph and display
grafo.show_buttons()
grafo.save_graph("IPC_Assignee_weighted_DW.html")

## Perguntas

### Quais oportunidades de patentes para serem usadas no Brasil em um setor X?

In [69]:
grafo = Network(height='800px', width='100%', notebook=True, cdn_resources='in_line')

for index, row in dw_df.iterrows():
    grafo.add_node(str(row["Patent Assignee Matrix"]), title=str(row["Patent Assignee Matrix"]),
                   color=row['circle_color'])
    grafo.add_node(str(row["International Patent Classification"]), title=str(row["IPC4_Desc"]),
                   color='red', shape='box')
    grafo.add_node(str(row["index"]), title=str(row["Patent Number"]),
                   color=row["triangle_color"], shape='triangle', size=row["Family Size"] * 7)

    # Add edge with weight as an attribute
    grafo.add_edge(str(row["Patent Assignee Matrix"]), str(row["index"]))
    grafo.add_edge(str(row["index"]), str(row["International Patent Classification"]))

grafo.show_buttons()
grafo.filter_menu = True
grafo.save_graph("IPC_Assignee_PN_DW.html")

In [70]:
# Grafo para aplicações WO
grafo = Network(height='800px', width='100%', notebook=True, cdn_resources='in_line')

for index, row in wo_df.iterrows():
    grafo.add_node(str(row["Patent Assignee Matrix"]), title=str(row["Patent Assignee Matrix"]),
                   color=row['circle_color'])
    grafo.add_node(str(row["International Patent Classification"]), title=str(row["IPC4_Desc"]), color='red', shape='box')
    grafo.add_node(str(row["Patent Number"]), title=str(row["Title"]), color=row["triangle_color"], shape='triangle', size=15)

    # Add edge with weight as an attribute
    grafo.add_edge(str(row["Patent Assignee Matrix"]), str(row["Patent Number"]))
    grafo.add_edge(str(row["Patent Number"]), str(row["International Patent Classification"]))

grafo.show_buttons()
grafo.filter_menu = True
grafo.save_graph("IPC_Assignee_PN_WO.html")

### Quais oportunidades de patentes em fim de vida para serem usadas no Brasil em um setor X?

As patentes em fim de vida (menos de 5 anos para expirar) estão marcadas em verde

### Quais empresas brasileiras combinam mais com o IPC destacado como oportunidade de mercado?

In [71]:
#df_ipcs_match = pd.read_csv("/content/drive/MyDrive/Projeto SENAI - Produções/Dados Energia/Sustainable Aviation Fuel - SAF/Patents_IPC.txt", sep=r'\t', engine='python')

In [72]:
#df_ipcs_match.rename(columns=patent_data, inplace=True)

In [73]:
#df_ipcs_match[df_ipcs_match["Patent Number"].str.contains("BR")]["Patent Number"].count()

### Qual é o IPC com maior número de arestas? Dentro desse IPC, qual a patente mais relevante?

In [74]:
# IPCs com maior quantidade de arestas
dw_df.groupby(["International Patent Classification", "IPC4_Desc"])["Patent Number"].nunique().sort_values(ascending=False).reset_index().head(10)

,International Patent Classification,IPC4_Desc,Patent Number
0,C10G,Cracking hydrocarbon oils; production of liqui...,22
1,F02C,Gas-turbine plants; air intakes for jet-propul...,18
2,C10L,Fuels not otherwise provided for; natural gas;...,14
3,B64D,Equipment for fitting in or to aircraft; fligh...,12
4,B01D,Separation (separating solids from solids by w...,8
5,B01J,"Chemical or physical processes, e.g. catalysis...",8
6,G01N,Investigating or analysing materials by determ...,8
7,C07C,Acyclic or carbocyclic compounds (macromolecul...,5
8,F02K,Jet-propulsion plants (arrangement or mounting...,5
9,C01B,Non-metallic elements; compounds thereof (ferm...,5


In [75]:
biggest_family = dw_df.groupby("International Patent Classification")[["Family Size"]].max()

In [76]:
# Elenca as maiores patentes dentro de um IPC
biggest_patent_from_IPC = pd.merge(dw_df, biggest_family, on=["International Patent Classification", "Family Size"], how="inner").groupby(["International Patent Classification", "Patent Number"])["Family Size"].max().sort_values(ascending=False).reset_index()

In [77]:
biggest_patent_from_IPC[biggest_patent_from_IPC["International Patent Classification"] == "C10G"]["Patent Number"].iloc[0]

'WO2022220246-A1;  JP2023514656-X;  CA3216230-A1;  KR2023170005-A;  CN117396582-A;  IN202347076905-A;  EP4324898-A1'

In [78]:
biggest_patent_from_IPC

,International Patent Classification,Patent Number,Family Size
0,F02C,US2023193835-A1; EP4202197-A1; AU2022283777-...,7
1,F02C,DE102022133283-A1; FR3130899-A1; US202319230...,7
2,G08G,US2023193835-A1; EP4202197-A1; AU2022283777-...,7
3,B64C,DE102022133283-A1; FR3130899-A1; US202319230...,7
4,B64D,DE102022133283-A1; FR3130899-A1; US202319230...,7
...,...,...,...
56,C08K,WO2022130772-A1; JP2022569739-X,1
57,C02F,US11472720-B1,1
58,B65G,IN202221002491-A,1
59,B60C,WO2022130772-A1; JP2022569739-X,1


In [79]:
dw_df.loc[dw_df.groupby(["International Patent Classification", "Patent Number"])["Family Size"].idxmax()].groupby(["International Patent Classification", "Patent Number"])["Family Size"].max().sort_values(ascending=False).reset_index()

,International Patent Classification,Patent Number,Family Size
0,F02C,DE102022133283-A1; FR3130899-A1; US202319230...,7
1,G08G,US2023193835-A1; EP4202197-A1; AU2022283777-...,7
2,B64D,DE102022133283-A1; FR3130899-A1; US202319230...,7
3,B64D,US2023193835-A1; EP4202197-A1; AU2022283777-...,7
4,B64C,DE102022133283-A1; FR3130899-A1; US202319230...,7
...,...,...,...
160,F02C,CN115355090-A,1
161,C10G,EP4197634-A1; WO2023111277-A2; WO2023111277-A3,1
162,F02B,CN117167133-A,1
163,F01K,IN202221002491-A,1


---